In [1]:
import numpy as np

In [2]:
class perceptron:
    """
    A simple implementation of a multi-layer perceptron (MLP) neural network.

    Attributes:
    - n_layers (int): Number of layers in the neural network.
    - n_neurons (list of int): Number of neurons in each layer.
    - neurons (list of Layer): List containing the layers of the neural network.
    - input (array): The input data for the forward pass.
    - output (array): The output prediction of the neural network.

    Methods:
    - __init__(self, n_inputs, n_neurons, n_activation): Initializes the perceptron.
    - forward(self, input): Performs the forward pass through the neural network.
    - backward(self, error): Performs the backward pass to update the network's weights.
    - train(self, x, Y, epochs): Trains the neural network on the given data.

    """

    def __init__(self, n_inputs, n_neurons, n_activation, eta = 1):
        """
        Initializes the multi-layer perceptron with the specified architecture.

        Args:
        - n_inputs (int): Number of input features.
        - n_neurons (list of int): Number of neurons in each layer.
        - n_activation (list): List of activation functions for each layer.

        """

        # Set the size of the network
        self.n_layers = len(n_neurons)
        self.n_neurons = n_neurons

        # Define an array to hold all the layers
        self.neurons = [Layer(n_inputs, n_neurons[0], n_activation[0], eta)]

        # Define all hidden layers
        for i in range(1, self.n_layers):
            self.neurons.append(Layer(n_neurons[i-1], n_neurons[i], n_activation[i], eta))

    def forward(self, input):
        """
        Performs the forward pass through the neural network.

        Args:
        - input (array): The input data for the forward pass.

        Returns:
        - output (array): The output prediction of the neural network.

        """

        # Save the input
        self.input = np.array(input)

        # Pass the input to the first layer
        self.neurons[0].forward(np.transpose(input))

        # Iterate through all neurons
        for i in range(1, self.n_layers):
            self.neurons[i].forward(self.neurons[i-1].output)

        # Set the MLP's output as the output of the last layer
        self.output = self.neurons[-1].output
        return self.output

    def backward(self, error):
        """
        Performs the backward pass to update the network's weights.

        Args:
        - error (array): The error in the prediction.

        Returns:
        - gradients_prom (list of float): List of average gradients for each layer.

        """
        #Initializing an array for the mean of the gradients in every layer
        gradients_mean = [0] * self.n_layers

        # Backpropagation for output layer
        local_gradient_prev = self.neurons[-1].backward(-np.sum(error), 1)
        gradients_mean[-1] = np.mean(local_gradient_prev)

        # Backpropagation for all the other layers
        for i in reversed(range(self.n_layers - 1)):
            local_gradient_prev = self.neurons[i].backward(self.neurons[i + 1].weights, local_gradient_prev)
            gradients_mean[i] = np.mean(local_gradient_prev)

        return gradients_mean


    def train(self, data, epochs, batch = False):
        """
        Trains the neural network on the given data using backpropagation.

        Args:
        - data (array): Input data with the last value the target column.
        - epochs (int): Number of training epochs.

        Returns:
        - gradients (list): List of gradients computed during training.

        """

        #This part of the code cannot yet withstand multiple outputs
        # Defining the train and test set
        train, test, _ = train_test_val(data, (75,25,0))
        train_x, train_y = train[:,0:-1], train[:,-1]
        test_x, test_y = test[:,0:-1], test[:,-1]

        # Check Y dimensions are of the size of the output layer
        #assert len(train_y[0]) == self.n_neurons[-1]
        
        # Define auxiliary variables
        data_len = len(train_x)
        gradients = [0] * data_len * epochs;  gradient_epochs = [0]*epochs 
        instant_energy_train = [0] * data_len * epochs;  instant_average_energy_train = [0]*epochs 
        instant_average_energy_test = [0]*epochs 
        counter = 0 

        for epoch in range(epochs):
            for i, input in enumerate(train_x):
              y_pred = self.forward(np.array([input]))
              error = np.array(train_y[i] - y_pred)
              instant_energy_train[counter] = np.sum(error**2)/2
              gradients[counter] = self.backward(error)
              counter += 1
            
            # Epoch relevant information
            gradient_epochs[epoch] = np.mean(gradients[epoch*data_len:epoch*data_len+data_len], axis = 0)
            instant_average_energy_train[epoch] = np.mean(instant_energy_train[epoch*data_len:epoch*data_len+data_len])

            #Test error
            error_test = test_y - self.forward(test_x)
            instant_average_energy_test[epoch] = np.mean(error_test**2)/2

            #print(error_p.shape, test_y.shape, self.forward(np.array(test_x)).shape)
              
        return gradients, gradient_epochs, instant_energy_train, instant_average_energy_train, instant_average_energy_test





In [3]:
import numpy as np

class Layer:
    """
    A class representing a single layer in a neural network.

    Attributes:
    - weights (array): Weight matrix for the layer's connections.
    - biases (array): Bias vector for the layer.
    - activation (Activation): The activation function for the layer.
    - stimuli (array): Input stimuli for the layer.
    - field (array): Linear combination of stimuli, weights, and biases.
    - output (array): Output of the layer after activation.
    - local_gradient (array): Local gradient used in backpropagation.

    Methods:
    - __init__(self, n_inputs, n_neurons, activation): Initializes the layer.
    - forward(self, stimuli): Performs the forward pass through the layer.
    - backward(self, weights_prev, local_gradient_prev): Performs backpropagation for the layer.

    """

    def __init__(self, n_inputs, n_neurons, activation, eta=1):
        """
        Initializes a neural network layer.

        Args:
        - n_inputs (int): Number of input features.
        - n_neurons (int): Number of neurons in the layer.
        - activation (Activation): Activation function for the layer.

        """

        # Initialize weights with random values
        self.weights = np.random.randn(n_neurons, n_inputs) * 2 - 1
        self.biases = np.zeros((n_neurons, 1))  # Initialize biases with zeros
        self.activation = activation()  # Create an instance of the provided activation function
        # Learning rate (you can adjust this)
        self.eta = eta

    def forward(self, stimuli):
        """
        Performs the forward pass through the layer.

        Args:
        - stimuli (array): The input stimuli for the layer.

        """

        # Save the input stimuli for later use in backward pass
        self.stimuli = stimuli

        # Compute the initial linear combination of stimuli, weights, and biases
        self.field = np.matmul(self.weights, self.stimuli) + self.biases

        # Apply the activation function to the linear combination
        self.output = self.activation.forward(self.field)

    def backward(self, weights_prev, local_gradient_prev):
        """
        Performs backpropagation for the layer.

        Args:
        - weights_prev (array): Weights from the next layer.
        - local_gradient_prev (array): Local gradient from the next layer.

        Returns:
        - local_gradient (array): Local gradient for this layer.

        """

        phi_prime = self.activation.backward(self.field)  # Compute the derivative of the activation function

        # Compute local gradient for this layer using chain rule and weights from the next layer
        self.local_gradient = np.multiply(phi_prime, np.dot(weights_prev.T, local_gradient_prev))

        # Compute weight update using the local gradient and input stimuli
        delta_weights = np.matmul(self.local_gradient, self.stimuli.T)
        assert delta_weights.shape == self.weights.shape

        # Update weights using the learning rate and calculated delta
        self.weights = self.weights + self.eta * delta_weights

        # Update biases using the learning rate and the local gradient
        delta_biases =  -self.eta * np.mean(self.local_gradient, axis=1, keepdims=True)
        assert delta_biases.shape == self.biases.shape
        self.biases = self.biases + self.eta *delta_biases

        return self.local_gradient


In [4]:
class sigmoid :
  def forward(self, input):
    self.output = 1/(1 + np.exp(-input))
    return self.output

  def backward(self, output):
    self.back = np.exp(-output)/(np.exp(-output) + 1)**2
    return self.back

class tanh :
  def forward(self, input):
    return np.tanh(input)

  def backward(self,output):
    self.back = 1 - np.tanh(output)**2
    return self.back

class linear:
  def forward(self, input):
    self.a = 1
    self.b = 0
    return np.dot(self.a,input) +self.b

  def backward(self, output):
    length = int(len(output))
    return np.array([[self.a]]*length)


In [5]:
# Data Manipulations

def train_test_val(data, sizes = (60,20,20)):
  sizes = [x/100 for x in sizes]
  length = len(data)
  nonTrainLen = int(length*(sizes[1]+sizes[2]))

  idx = np.random.choice(np.arange(0,length), size = nonTrainLen, replace=False)
  assert len(idx) == nonTrainLen

  idxTrain = np.setdiff1d(np.arange(0,length), idx)
  assert len(idx) == nonTrainLen

  testLen = int(nonTrainLen*sizes[1]/(sizes[1]+sizes[2]))
  idxTest = np.random.choice(np.arange(0,len(idx)), size = testLen, replace=False)
  assert len(idxTest) == testLen

  testVal = int(nonTrainLen*sizes[2]/(sizes[1]+sizes[2]))
  idxValid = np.setdiff1d(np.arange(0,nonTrainLen), idxTest)
  assert len(idxValid) == testVal

  assert(list(np.intersect1d(idxValid, idxTest)) == [])
  assert(list(np.intersect1d(idx, idxTrain)) == [])
  train = np.array([data[i] for i in idxTrain])
  test, valid = np.array([data[idx[i]] for i in idxTest]), np.array([data[idx[i]] for i in idxValid])
  return train, test, valid

def normalize_min_max(matrix):
  min_val = np.min(matrix)
  max_val = np.max(matrix)
  normalized_matrix = (matrix - min_val) / (max_val - min_val)
  return normalized_matrix

### DatosIA.mat

In [6]:
import scipy.io

mat = scipy.io.loadmat('datosIA.mat')

data = np.column_stack([mat["X"], mat["OD"], mat["S"]])
data = normalize_min_max(data)

# X = [X, OD], Y = [S]
train, _, valid = train_test_val(data, (80,0,20))

n_neurons = [2,1]
n_activation = [sigmoid]*len(n_neurons)

MLP_1CapaOculta = perceptron(2,n_neurons, n_activation,0.1)


In [179]:
gradients, gradient_epochs, instant_energy_train, instant_average_energy_train, instant_average_energy_test = MLP_1CapaOculta.train(train,20)

In [180]:
instant_average_energy_train

[0.05041685984414076,
 0.051851222159427564,
 0.05295970502774779,
 0.053944788521051515,
 0.05493313962478981,
 0.055968773751773594,
 0.056997976548206736,
 0.05789069380547213,
 0.05852028637277819,
 0.058845636115228056,
 0.05892146006771794,
 0.058845254262945,
 0.058703809532889215,
 0.05855222129115055,
 0.05841721126812222,
 0.058307726532040305,
 0.05822384298999303,
 0.05816212970354628,
 0.058118321140156026,
 0.05808844241657907]

In [183]:
[1] + [2,3,5,3] + [3]

[1, 2, 3, 5, 3, 3]